In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

input_shape = x_train.shape[1:]

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [5]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', 
                       kernel_initializer='lecun_normal', 
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus, 
                       kernel_initializer=my_glorot_initializer, 
                       kernel_regularizer=my_l1_regularizer, 
                       kernel_constraint=my_positive_weights)
])

In [6]:
model.compile(loss='mse', optimizer='nadam', metrics=['mae'])

In [7]:
model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 873us/step - loss: 1.5542 - mae: 0.8962 - val_loss: 1.4154 - val_mae: 0.5607
Epoch 2/2
363/363 [==============================] - 0s 649us/step - loss: 0.5943 - mae: 0.5256 - val_loss: 1.4399 - val_mae: 0.5137


In [8]:
model.save('model_with_custom_parts.h5')

In [9]:
model = keras.models.load_model('model_with_custom_parts.h5', 
                                custom_objects={                                    
                                    'my_softplus': my_softplus,
                                    'my_glorot_initializer': my_glorot_initializer,
                                    'my_l1_regularizer': my_l1_regularizer,
                                    'my_positive_weights': my_positive_weights,
                                })

In [10]:
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {'factor': self.factor}

In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', 
                       kernel_initializer='lecun_normal', 
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus, 
                       kernel_initializer=my_glorot_initializer, 
                       kernel_regularizer=MyL1Regularizer(0.01), 
                       kernel_constraint=my_positive_weights)
])

In [13]:
model.compile(loss='mse', optimizer='nadam', metrics=['mae'])

In [14]:
model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 809us/step - loss: 1.5542 - mae: 0.8962 - val_loss: 1.4154 - val_mae: 0.5607
Epoch 2/2
363/363 [==============================] - 0s 641us/step - loss: 0.5943 - mae: 0.5256 - val_loss: 1.4399 - val_mae: 0.5137


In [15]:
model.save('model_with_many_custom_parts.h5')

In [16]:
model = keras.models.load_model('model_with_custom_parts.h5', 
                                custom_objects={                                    
                                    'my_softplus': my_softplus,
                                    'my_glorot_initializer': my_glorot_initializer,
                                    'my_l1_regularizer': MyL1Regularizer(0.01),
                                    'my_positive_weights': my_positive_weights,
                                })